In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
# df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
902,The film disappointed me for many reasons: fir...,negative
915,"the film itself is absolutely brilliant, its t...",positive
426,Well you take O.J. Simpson as a all american s...,positive
388,Ostensibly a story about the young child of Ji...,positive
664,As always Joan Hickson is wonderful as Jane M....,positive


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
902,film disappointed many reason first depiction ...,negative
915,film absolutely brilliant buzz rush make want ...,positive
426,well take o j simpson american soldier turned ...,positive
388,ostensibly story young child jimmy stewart dor...,positive
664,always joan hickson wonderful jane m subtle sh...,positive


In [5]:
df['sentiment'].value_counts()

sentiment
negative    271
positive    229
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
902,film disappointed many reason first depiction ...,0
915,film absolutely brilliant buzz rush make want ...,1
426,well take o j simpson american soldier turned ...,1
388,ostensibly story young child jimmy stewart dor...,1
664,always joan hickson wonderful jane m subtle sh...,1


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
mlflow_tracking_uri = os.environ.get("MLFLOW_TRACKING_URI")
dagshub_cred_owner = os.environ.get("dagshub_cred_owner")
dagshub_cred_name = os.environ.get("dagshub_cred_name")

In [ ]:
import dagshub
mlflow.set_tracking_uri(mlflow_tracking_uri)
dagshub.init(repo_owner=dagshub_cred_owner, repo_name=dagshub_cred_name , mlflow=True)


mlflow.set_experiment("Logistic Regression Baseline")


2025-09-03 22:56:42,144 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/sa-rehman1/Complete_ML_Pipeline_Project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "sa-rehman1/Complete_ML_Pipeline_Project"

2025-09-03 22:56:42,147 - INFO - Initialized MLflow to track repo "sa-rehman1/Complete_ML_Pipeline_Project"


Repository sa-rehman1/Complete_ML_Pipeline_Project initialized!

2025-09-03 22:56:42,149 - INFO - Repository sa-rehman1/Complete_ML_Pipeline_Project initialized!


<Experiment: artifact_location='mlflow-artifacts:/1ffc37cdd9304eefa4fe012789d7fe2a', creation_time=1756957827845, experiment_id='0', last_update_time=1756957827845, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [15]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-09-03 22:56:43,531 - INFO - Starting MLflow run...
2025-09-03 22:56:43,701 - INFO - Logging preprocessing parameters...
2025-09-03 22:56:44,129 - INFO - Initializing Logistic Regression model...
2025-09-03 22:56:44,129 - INFO - Fitting the model...
2025-09-03 22:56:44,142 - INFO - Model training complete.
2025-09-03 22:56:44,142 - INFO - Logging model parameters...
2025-09-03 22:56:44,262 - INFO - Making predictions...
2025-09-03 22:56:44,264 - INFO - Calculating evaluation metrics...
2025-09-03 22:56:44,268 - INFO - Logging evaluation metrics...
2025-09-03 22:56:44,823 - INFO - Saving and logging the model...
2025/09/03 22:56:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-09-03 22:56:49,892 - INFO - Model training and logging completed in 6.19 seconds.
2025-09-03 22:56:49,892 - INFO - Accuracy: 0.62
2025-09-03 22:56:49,894 - INFO - Precision: 0.

🏃 View run big-shrike-624 at: https://dagshub.com/sa-rehman1/Complete_ML_Pipeline_Project.mlflow/#/experiments/0/runs/7947c0a2e32748c0bbe211c7782e380a
🧪 View experiment at: https://dagshub.com/sa-rehman1/Complete_ML_Pipeline_Project.mlflow/#/experiments/0
